<a href="https://colab.research.google.com/github/wdempsey/AI4Health-Online-Experimentation/blob/main/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online learning and experimentation algorithms in mobile health

In [2]:
## Import necessary 
import numpy as np

#Part 1: Overview on Linear Contextual Bandits


At each decision time $t$,  suppose we observe a state variable $S_t \in \mathbb{R}^p$

$$
R_t = r(C_t, A_t) + \epsilon_t
$$

where $




In [ ]:
# Simulation example

## reward is Simply



In [3]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
print(s)


[-9.65722332e-02  9.69156929e-02  9.42467366e-02 -8.50898674e-02
  1.10717523e-01  8.95319185e-02  8.65105747e-03 -5.81047649e-02
  1.11961890e-01 -5.46428471e-02 -8.87028091e-02  1.10609876e-01
 -5.02678903e-02  6.74152130e-02  7.23218813e-02 -3.65010924e-02
 -4.41046330e-02 -3.54904886e-02  1.25087733e-01 -5.97111148e-04
  2.23295805e-02 -1.99769763e-02 -3.69629448e-03 -7.38182235e-02
 -3.22056077e-03 -6.08815779e-02 -1.54906874e-01 -2.11492919e-01
  7.36109202e-03 -1.19107611e-01  1.29090163e-02  1.54581538e-01
  1.49865976e-01  3.84180785e-02  1.08864488e-02 -1.31393853e-01
  1.60316607e-01 -5.77178070e-02  1.08702906e-01  9.42327898e-02
 -1.89725684e-01  1.17138702e-01 -1.38425999e-01 -9.92822162e-02
  6.65458215e-02  9.89079267e-02 -2.87391062e-02  5.28174234e-02
 -2.36612687e-03 -2.40754185e-02 -1.77969372e-02 -1.91995374e-02
  2.06979018e-02  1.72081783e-01 -8.46914015e-02  8.03001927e-02
  6.42488068e-02  2.73727375e-02 -7.35016432e-02 -1.87389934e-01
  2.44659737e-02  7.04339

## Running example: HeartSteps V2



# Part 2: Constrained Optimization